In [13]:
import torch
from torchvision import transforms
import os 
from torch.utils.data import DataLoader, Dataset
from PIL import Image

from torch import nn
from torch.nn import functional as F

In [14]:
class Loader(Dataset):
    def __init__(self, split, trasform = None):
        base_dir='./datasets/'

        path =os.path.join(base_dir,'64x64_SIGNS/{}'.format(split))

        files = os.listdir(path)

        self.filenames = [os.path.join(path,f) for f in files if f.endswith('.jpg')]

        self.targets = [int(f[0]) for f in files]

        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        image = Image.open(self.filenames[idx])

        if self.transform:
            image = self.transform(image)
        
        return image, self.targets[idx]

In [15]:

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [16]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=9,kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=9,out_channels=12,kernel_size=5)
        self.fc1 = nn.Linear(2028,1000)
        self.fc2 = nn.Linear(1000,500)
        self.fc3 = nn.Linear(500,6)
        self.pool = nn.MaxPool2d(2)

    def forward(self,x):
        x = self.pool(F.relu( self.conv1(x)))
        x = self.pool(F.relu( self.conv2(x)))

        x = x.view(-1,2028)

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        x = F.log_softmax(x,dim=1)
        return x
net = Net()

In [17]:
testSet = Loader('test_signs',transform)


In [18]:
with torch.no_grad():
    mistakes = 0
    for i in range(100):
        net.load_state_dict(torch.load('borrar.pth'))
        image, label = testSet[i]
        image = torch.tensor(image, dtype= torch.float32).unsqueeze(0)
        out = net(image)
        porcent = torch.nn.functional.softmax(out, dim=1)[0]*100
        _, index = torch.max(out, dim=1)
        print('input: ',label,'    output: ',index.item(),'   acuracy: ',porcent[index[0]].item())
        if label != index.item():
            mistakes +=1

input:  5     output:  5    acuracy:  100.0
input:  5     output:  5    acuracy:  100.0
input:  5     output:  5    acuracy:  100.0
input:  0     output:  0    acuracy:  100.0
input:  4     output:  4    acuracy:  100.0
input:  1     output:  1    acuracy:  100.0
input:  5     output:  5    acuracy:  100.0
input:  0     output:  0    acuracy:  100.0
input:  4     output:  2    acuracy:  95.74536895751953
input:  2     output:  2    acuracy:  100.0
input:  4     output:  4    acuracy:  100.0
input:  0     output:  0    acuracy:  100.0
input:  2     output:  2    acuracy:  100.0
input:  5     output:  5    acuracy:  100.0
input:  4     output:  4    acuracy:  100.0
input:  5     output:  5    acuracy:  100.0
input:  0     output:  0    acuracy:  100.0
input:  3     output:  5    acuracy:  99.99991607666016
input:  3     output:  3    acuracy:  100.0
input:  5     output:  5    acuracy:  99.99996185302734
input:  2     output:  2    acuracy:  99.43107604980469
input:  5     output:  5    

In [19]:
mistakes

8